In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
from trackml.dataset import load_event
from trackml.randomize import shuffle_hits
from trackml.score import score_event

In [3]:
import os
import numpy as np
import pandas as pd
import glob
import math

import sys
sys.path.insert(0, '../')

import time
from utils import timeSince
from tqdm import tqdm

In [4]:

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
%matplotlib inline


In [5]:
from rnn_module_id import ModuleIDBasedRNN
from rnn_module_id import load_training
from rnn_module_id import input_target
from rnn_module_id import modules

In [6]:
from utils import tunable_parameters

In [7]:
model = ModuleIDBasedRNN(input_dim=modules+1,
                         hidden_dim=20,
                         output_dim=modules+1,
                         batch_size=1)

In [8]:
train_data = load_training()
print("Total events:", len(train_data))
print("Total truth particles:", sum([y.shape[0] for x,y in train_data]))

Total events: 1770
Total truth particles: 14496495


In [9]:
event, truth = train_data[1000]

In [10]:
print(event.shape)
print(truth.shape)
print(type(event))
print(type(truth))

(97565, 2)
(8931,)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [14]:
hits = event[event['particle_id'] == truth.values[0]]['uID'].values

In [ ]:
print(len(hits))
print(hits)

In [21]:
def predict(model, input, batch_size=1):
    with torch.no_grad():
        
        model.hidden = model.init_hidden()
        output = model(input)
        print(output.size())
        topv, topi = output.topk(3)
        return topi
#         cand = topi[0][0]
#         return cand.item()

In [16]:
input, target = input_target(hits)

In [17]:
model_input = 'output/model/RNNModule_1'
model.load_state_dict(torch.load(model_input, map_location=lambda storage, loc: storage))

In [22]:
topi = predict(model, input)

torch.Size([11, 18729])


In [23]:
print(topi.size())

torch.Size([11, 3])


In [24]:
print(topi)

tensor([[   883,    867,    881],
        [  1213,   1211,   1235],
        [ 18728,   1762,   1788],
        [ 18728,   5401,   5471],
        [ 18728,   5401,   5417],
        [ 18728,  10862,   5417],
        [ 18728,  11013,  10994],
        [ 18728,   4918,   4865],
        [ 18728,   4720,   4717],
        [ 18728,   4717,   4757],
        [ 18728,   4717,   4757]])
